## Preprocessing

In [1]:
import pandas as pd
import re

df = pd.read_csv('C:/Users/Ale/Desktop/jupyter_folder/train.csv')
df.fillna('', inplace=True)

df

id                                        raw_address  \
0            0  jl kapuk timur delta sili iii lippo cika 11 a ...   
1            1                                 aye, jati sampurna   
2            2               setu siung 119 rt 5 1 13880 cipayung   
3            3                               toko dita, kertosono   
4            4                                      jl. orde baru   
...        ...                                                ...   
299995  299995               jend ahmad yani 331 kertasari ciamis   
299996  299996                 raya cila kko, cilandak timur kel.   
299997  299997                     tanjung gusta jl. yaya 2 no 17   
299998  299998  jalan cipadu jaya taman asri gang bijaksana 3 ...   
299999  299999          jaya maspion permata blok beryl b2  no.58   

                                       POI/street  
0       /jl kapuk timur delta sili iii lippo cika  
1                                               /  
2                                          /siung  
3                                      toko dita/  
4                                  /jl. orde baru  
...                                           ...  
299995                           /jend ahmad yani  
299996                             /raya cila kko  
299997                                          /  
299998                                taman asri/  
299999                      jaya maspion permata/  

[300000 rows x 3 columns]

In [ ]:
!pip install spacy==2.3.5

In [2]:
import string

TRAIN_COMBINED = []

total_conflicts = 0

def find_match(raw_words, target_words, ignore_truncated=True):
    
    low = 0
    high = 0
    pattern_index = 0
    found = False
            
    while high < len(raw_words):

        word = raw_words[high]

        if pattern_index == len(target_words):
            found = True
            break
            
        elif not ignore_truncated and (target_words[pattern_index].startswith(word) or target_words[pattern_index].startswith(word[:-1])):
            high += 1
            pattern_index += 1

        elif target_words[pattern_index] == word or target_words[pattern_index] == word[:-1]:
            high += 1
            pattern_index += 1

        else:
            low += 1
            high = low
            pattern_index = 0
            
    if pattern_index == len(target_words):
        found = True

    if found:
        vector = [1 if i >= low and i < high else 0 for i in range(len(raw_words))]
    else:
        vector = [0 for i in range(len(raw_words))]
        
    return vector

for index, row in df.iterrows():
    raw_address = row['raw_address']
    poi, street = row['POI/street'].split('/')
    
    combined_entities = []
    
    first_entity = None
    
    if poi:
        
        m = re.search(re.escape(poi), raw_address)
        
        if m:
            low, high = m.start(), m.end()
        
        else:
            
            vector = find_match(raw_address.split(), poi.split(), False)

            curr_idx = 0
            low = None
            high = None

            for i in range(len(vector)):
                if vector[i] == 1 and low == None:
                    low = curr_idx
                elif vector[i] == 0 and low != None and high == None:
                    high = curr_idx - 1
                    break

                curr_idx += len(raw_address.split()[i]) + 1

            if low != None and high == None:
                high = len(raw_address)

        if low != None and high != None:

            while raw_address[high - 1] in string.punctuation:
                high -= 1

            first_entity = (low, high, "POI")
            combined_entities.append((low, high, "POI"))
            
    if street:
        
        m = re.search(re.escape(street), raw_address)
        
        if m:
            low, high = m.start(), m.end()
        
        else:
        
            vector = find_match(raw_address.split(), street.split(), False)

            curr_idx = 0
            low = None
            high = None

            for i in range(len(vector)):
                if vector[i] == 1 and low == None:
                    low = curr_idx
                elif vector[i] == 0 and low != None and high == None:
                    high = curr_idx - 1
                    break

                curr_idx += len(raw_address.split()[i]) + 1

            if low != None and high == None:
                high = len(raw_address)
        
        if low != None and high != None:
            
            while raw_address[high - 1] in string.punctuation:
                high -= 1
            
            if first_entity and (
                # Starting index overlaps with POI tag
                low >= first_entity[0] and low <= first_entity[1] or 
                
                # Ending index overlaps with POI tag
                high >= first_entity[0] and high <= first_entity[1] or
                
                # POI tag is a subset of SRT tag
                low < first_entity[0] and high > first_entity[1]
            ):
                # Conflicting tags
                total_conflicts += 1
            
            else:
                combined_entities.append((low, high, "SRT"))
    
    combined_spacy_entry = (raw_address, {"entities": combined_entities})
    TRAIN_COMBINED.append(combined_spacy_entry)

print(f"{total_conflicts} total conflicts.")
print(TRAIN_COMBINED[:10])

941 total conflicts.
[('jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat', {'entities': [(0, 40, 'SRT')]}), ('aye, jati sampurna', {'entities': []}), ('setu siung 119 rt 5 1 13880 cipayung', {'entities': [(5, 10, 'SRT')]}), ('toko dita, kertosono', {'entities': [(0, 9, 'POI')]}), ('jl. orde baru', {'entities': [(0, 13, 'SRT')]}), ('raya samb gede, 299 toko bb kids', {'entities': [(20, 32, 'POI'), (0, 14, 'SRT')]}), ('kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu', {'entities': [(0, 12, 'SRT')]}), ('tela keuramat kuta alam', {'entities': [(0, 4, 'SRT')]}), ('gg. i wates magersari', {'entities': [(0, 5, 'SRT')]}), ('bunga ncole ix 2', {'entities': [(0, 14, 'SRT')]})]


## Training

In [3]:
import spacy
import random
import logging
logging.captureWarnings(True)

print("Preparing...")

nlp = spacy.blank("id") # Indonesian
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("POI")
ner.add_label("SRT")

Preparing...


In [4]:
from spacy.util import minibatch, compounding
from spacy.util import decaying

print("Training...")

# Start the training
nlp.begin_training()

dropout = decaying(0.6, 0.2, 1e-4)

# Loop for 20 iterations
for itn in range(20):
    
    # Shuffle the training data
    random.shuffle(TRAIN_COMBINED)
    losses = {}
    
    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAIN_COMBINED, size=compounding(4.0, 32.0, 1.001)):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]
        
        # Update the model
        nlp.update(texts, annotations, losses=losses, drop=next(dropout))
        
    print(f"Epoch: {itn}\tLosses: {str(losses)}")
    
nlp.to_disk("combined.model")

Training...
Epoch: 0	Losses: {'ner': 297054.9482986609}
Epoch: 1	Losses: {'ner': 216277.66637733497}
Epoch: 2	Losses: {'ner': 196077.51304265475}
Epoch: 3	Losses: {'ner': 185385.94019547015}
Epoch: 4	Losses: {'ner': 177719.78595339146}
Epoch: 5	Losses: {'ner': 171903.05622700785}
Epoch: 6	Losses: {'ner': 167067.78700522718}
Epoch: 7	Losses: {'ner': 163160.0341156639}
Epoch: 8	Losses: {'ner': 160294.108030776}
Epoch: 9	Losses: {'ner': 157781.3684608443}
Epoch: 10	Losses: {'ner': 155059.1300248881}
Epoch: 11	Losses: {'ner': 152493.615438676}
Epoch: 12	Losses: {'ner': 151733.1806570454}
Epoch: 13	Losses: {'ner': 149802.21702654063}
Epoch: 14	Losses: {'ner': 147969.65492597746}
Epoch: 15	Losses: {'ner': 146884.2399998252}
Epoch: 16	Losses: {'ner': 144916.81584576552}
Epoch: 17	Losses: {'ner': 144169.03249486495}
Epoch: 18	Losses: {'ner': 142672.7668120506}
Epoch: 19	Losses: {'ner': 142475.8033859096}
